# Energy Calculation

In [8]:
import pandas as pd
import numpy as np


In [4]:
# Read DataFrame
pkls_path = pkls = r'D:\Tesis\ResEleCon-MX\pickles'
enigh = pd.read_pickle(pkls_path + '\enigh_fullMX.pkl')

## Tariff from CFE database

In [9]:
# Read cfe totals per municipality and tariff
cols = list(range(0,6))+[13]
CFE_mun = pd.read_csv("D:/Tesis/Datos/Cosumo por sectores por año/consumodeelectricidadpormunicipio.csv", usecols=cols, header=2)

CFE_mun['2017'] = (CFE_mun['2017'].replace({',': ''}, regex=True)).astype(np.int64)

# Assign corresponding ZM to each municipality
CFE_mun['CVE_MUN'] = (CFE_mun['Cve Inegi']*1000 + CFE_mun['Cve Mun'])
#CFE_mun = CFE_mun.merge(ZM_2015[['CVE_MUN','CVE_ZM']], on='CVE_MUN')

# Filter Residential consumption
domestic = ['1','1A','1B','1C','1D','1E','1F','1F','DAC']
CFE_mun['Tarifa'] = CFE_mun['Tarifa'].str.strip()
CFE_res = CFE_mun.loc[CFE_mun['Tarifa'].isin(domestic)]

# Select tariff with highest consumption
tarifa_cfe = CFE_res[['CVE_MUN','Tarifa']].loc[CFE_res.groupby('CVE_MUN')['2017'].idxmax()]
enigh_tar = enigh.merge(tarifa_cfe, on='CVE_MUN', how='left')

In [18]:
enigh_tar.head()

,folioviv,foliohog,CVE_MUN,estado,publico,CVE_ZM,NOM_ZM,CVE_ENT,NOM_ENT,NOM_MUN,...,gasto_tri_lpg,gasto_tri_oil,gasto_tri_diesel,gasto_tri_coal,gasto_tri_wood,gasto_tri_heat,gasto_tri_Magna,gasto_tri_Premium,gasto_tri_Die-Gas,Tarifa
0,100013601,1,1001,1,0.00,1.01,Aguascalientes,1.0,Aguascalientes,Aguascalientes,...,NaN,NaN,NaN,NaN,NaN,NaN,7200.0,NaN,NaN,1
1,100013602,1,1001,1,578.56,1.01,Aguascalientes,1.0,Aguascalientes,Aguascalientes,...,NaN,NaN,NaN,NaN,NaN,NaN,3600.0,NaN,NaN,1
2,100013603,1,1001,1,4255.68,1.01,Aguascalientes,1.0,Aguascalientes,Aguascalientes,...,2100.0,NaN,NaN,NaN,NaN,NaN,NaN,4500.0,NaN,1
3,100013604,1,1001,1,0.00,1.01,Aguascalientes,1.0,Aguascalientes,Aguascalientes,...,NaN,NaN,NaN,NaN,NaN,NaN,6000.0,NaN,NaN,1
4,100013606,1,1001,1,0.00,1.01,Aguascalientes,1.0,Aguascalientes,Aguascalientes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


## Tarifas CFE

### DAC Regions
Fot domestic high consumption (DAC) the applied tariff depends on the a different spatial distribution. The regions are: Baja California, Baja California Sur, Centro, Noroeste, Norte-Noreste y Sur-Peninsular.
https://energypedia.info/images/f/fb/GIZ_Tutorial_Tarifas_El%C3%A9ctricas_2015.pdf

In [19]:
# Region
list_edomex = [15108,15109,15039,15025,15037,15076,15106,15012,15024,15020,15121,15013,15104,15057,15033,15031,15029,15099,17007]
list_NE = [30123,30133,24011,24024]

conditions = [
    (enigh_tar['estado']==2),
    (enigh_tar['estado']==3),
    (enigh_tar['estado']==9) | (enigh_tar['CVE_MUN'].isin(list_edomex)),
    (enigh_tar['estado'].isin([25,26])),
    (enigh_tar['estado'].isin([5,8,10,19,28])) | (enigh_tar['CVE_MUN'].isin(list_NE)),
    (enigh_tar['estado'].isin([1,4,6,7,11,12,13,14,15,16,17,18,20,21,22,23,24,27,29,30,31,32])) & (~enigh_tar['CVE_MUN'].isin(list_NE+list_edomex))]
regiones = ['BC','BCS','Ce','NO','NNE','SP']
enigh_tar['Region'] = np.select(conditions, regiones, default=np.nan)

### Tariff
The prices taken are those of 2018 and the months from June to Octuber. This because the survey was carried out between August 21th and November 28th in 2018. As there are no information of the exact dates for the surveys, the values of the different months were averaged, so were the values of summer and non-summer months.

- 1-1F: https://app.cfe.mx/Aplicaciones/CCFE/Tarifas/Tarifas/tarifas_casa.asp
- DAC:  https://app.cfe.mx/Aplicaciones/CCFE/Tarifas/Tarifas/tarifas_casa.asp?Tarifa=DAC2003&Anio=2016&mes=7&imprime=

In [30]:
tarifas = ['1','1A','1B','1C','1D','1E','1F']

# Create df for tariffs
cfe_dic = {'tar': tarifas, 
           'Basico': [0.793,0.793,0.793,0.793,0.793,0.793,0.793], 
           'BasicoV': [np.nan,0.697,0.697,0.697,0.697,0.583,0.583,],
           'Intermedio': [0.956,0.956,0.956,0.956,0.956,0.956,0.956], 
           'IntermedioV': [np.nan,0.822,0.822,0.822,0.822,0.726,0.726],
           'IntermedioVA': [np.nan,np.nan,np.nan,1.05,1.05,0.948,1.768], 
           'Excedente': [2.802,2.802,2.802,2.802,2.802,2.802,2.802],
           'LimB': [75,75,75,75,75,75,75], 
           'LimBV': [np.nan,100,125,150,175,300,300], 
           'LimI': [140,150,175,175,200,200,200],
           'LimIV': [np.nan,150,225,300,400,750,1200], 
           'LimIVA': [np.nan,np.nan,np.nan,450,600,900,2500], 
           'Ldac': [250,300,400,850,1000,2000,2500]
          }

cfe = pd.DataFrame(data=cfe_dic)

cfe['LB_pesos'] = cfe['Basico']*cfe['LimB']
cfe['LBV_pesos'] = cfe['BasicoV']*cfe['LimBV']
cfe['LI_pesos'] = (cfe['LimI']-cfe['LimB'])*cfe['Intermedio']+cfe['LB_pesos']
cfe['LIV_pesos'] = (cfe['LimIV']-cfe['LimBV'])*cfe['IntermedioV']+cfe['LBV_pesos']
cfe['LIVA_pesos'] = (cfe['LimIVA']-cfe['LimIV'])*cfe['IntermedioVA']+cfe['LIV_pesos']
cfe['Ldac_pesos'] = (cfe['Ldac']-cfe['LimI'])*cfe['Excedente']+cfe['LI_pesos']
cfe.loc[0:3,'LdacV_pesos'] = (cfe['Ldac']-cfe['LimIV'])*cfe['Excedente']+cfe['LIV_pesos']
cfe.loc[3:,'LdacV_pesos'] = (cfe['Ldac']-cfe['LimIVA'])*cfe['Excedente']+cfe['LIVA_pesos']

cfe.set_index('tar', inplace=True)

# Create df for regions
dac_dic = {'Region': ['BC','BCS','Ce','NO','NNE','SP'], 
           'c_fijo_1': [100.63]*6, 'c_fijo_2': [100.77]*6, 'c_fijo_3': [102.24]*6, 'c_fijo_4': [102.45]*6, 'c_fijo_5': [102.79]*6, 'c_fijo_6': [102.80]*6,
           'c_fijo_7': [104.77]*6, 'c_fijo_8': [106.68]*6, 'c_fijo_9': [105.62]*6, 'c_fijo_10': [105.17]*6, 'c_fijo_11': [105.64]*6, 'c_fijo_12': [105.82]*6,
           'c_ener_1': [3.75,3.75,4.49,4.20,4.10,4.16], 'c_ener_2': [3.81,3.81,4.56,4.27,4.16,4.23], 'c_ener_3': [3.96,3.96,4.74,4.44,4.33,4.40], 'c_ener_4': [3.79,3.79,4.54,4.25,4.15,4.21], 'c_ener_5': [3.79,3.79,4.54,4.25,4.14,4.21], 'c_ener_6': [3.99,3.99,4.78,4.47,4.36,4.43],
           'c_ener_7': [4.08,4.08,4.89,4.58,4.46,4.53], 'c_ener_8': [4.11,4.11,4.92,4.61,4.49,4.56], 'c_ener_9': [3.97,3.97,4.75,4.45,4.34,4.41], 'c_ener_10': [4.01,4.01,4.81,4.50,4.39,4.46], 'c_ener_11': [4.13,4.13,4.95,4.64,4.52,4.59], 'c_ener_12': [4.21,4.21,5.04,4.72,4.60,4.67],
           
           'c_ener_v_1': [4.36,4.75,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_2': [4.43,4.83,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_3': [4.61,5.02,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_4': [4.41,4.81,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_5': [4.41,4.81,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_6': [4.64,5.06,np.nan,np.nan,np.nan,np.nan],
           'c_ener_v_7': [4.75,5.18,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_8': [4.78,5.21,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_9': [4.62,5.03,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_10': [4.67,5.09,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_11': [4.81,5.25,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_12': [4.90,5.34,np.nan,np.nan,np.nan,np.nan]
            }
dac = pd.DataFrame(data=dac_dic)

dac['c_fijo_v_m'] = dac.loc[:,'c_fijo_5':'c_fijo_10'].mean(axis=1)
dac['c_fijo_m'] = dac[['c_fijo_1','c_fijo_2','c_fijo_3','c_fijo_4','c_fijo_11','c_fijo_12']].mean(axis=1)
dac['c_ener_v_m'] = dac.loc[:,'c_ener_v_5':'c_ener_v_10'].mean(axis=1)
dac['c_ener_m'] = dac[['c_ener_1','c_ener_2','c_ener_3','c_ener_4','c_ener_11','c_ener_12']].mean(axis=1)
dac['c_ener_m_sm'] = dac.loc[:,'c_ener_5':'c_ener_10'].mean(axis=1) # Non-summer special tariff for summer months
dac.loc[0:1,'c_ener_m_sm'] = dac.loc[0:1,'c_ener_v_m']
dac.set_index('Region', inplace=True)

## MXN to kWh

In [35]:
# Take rows of CFE's costumers
ele = enigh_tar.loc[(enigh_tar['disp_elect']==1)].copy() #& (~ZM_gi['gasto_tri_ele'].isnull())
tol_dac = 1 # tolerance for DAC limit

# Convert pesos to kWh
def mxn_to_kwh(gasto, tarifa, region, tipo_tar = False):
    gasto = gasto/1.16
    
    #not summer
    if gasto <= cfe.loc[tarifa,'LB_pesos']:
        kwh_nv = gasto/cfe.loc[tarifa,'Basico']
    elif gasto <= cfe.loc[tarifa,'LI_pesos']:
        kwh_nv = (gasto - cfe.loc[tarifa,'LB_pesos'])/cfe.loc[tarifa,'Intermedio'] + cfe.loc[tarifa,'LimB']
    #elif gasto <= cfe.loc[tarifa,'Ldac_pesos']*tol_dac:
    elif gasto <= (cfe.loc[tarifa,'Ldac']*dac.loc[region,'c_ener_m']+dac.loc[region,'c_fijo_m'])*tol_dac:
        kwh_nv = (gasto - cfe.loc[tarifa,'LI_pesos'])/cfe.loc[tarifa,'Excedente'] + cfe.loc[tarifa,'LimI']
    else:
        kwh_nv = (gasto - dac.loc[region,'c_fijo_m'])/dac.loc[region,'c_ener_m']
    
    #summer
    if (tarifa == '1') & (gasto <= (cfe.loc[tarifa,'Ldac']*dac.loc[region,'c_ener_m_sm']+dac.loc[region,'c_fijo_v_m'])*tol_dac):
        kwh_v = kwh_nv
    elif (tarifa == '1') & (gasto > (cfe.loc[tarifa,'Ldac']*dac.loc[region,'c_ener_m_sm']+dac.loc[region,'c_fijo_v_m'])*tol_dac):
        kwh_v = (gasto - dac.loc[region,'c_fijo_v_m'])/dac.loc[region,'c_ener_m_sm']
    else:    
        if gasto <= cfe.loc[tarifa,'LBV_pesos']:
            kwh_v = gasto/cfe.loc[tarifa,'BasicoV']
        elif gasto <= cfe.loc[tarifa,'LIV_pesos']:
            kwh_v = (gasto - cfe.loc[tarifa,'LBV_pesos'])/cfe.loc[tarifa,'IntermedioV'] + cfe.loc[tarifa,'LimBV']
        elif gasto <= cfe.loc[tarifa,'LIVA_pesos']:
            kwh_v = (gasto - cfe.loc[tarifa,'LIV_pesos'])/cfe.loc[tarifa,'IntermedioVA'] + cfe.loc[tarifa,'LimIV']
        #elif gasto <= cfe.loc[tarifa,'LdacV_pesos']*tol_dac:
        elif gasto <= (cfe.loc[tarifa,'Ldac']*dac.loc[region,'c_ener_m_sm']+dac.loc[region,'c_fijo_v_m'])*tol_dac:
            if np.isnan(cfe.loc[tarifa,'LIVA_pesos']):
                kwh_v = (gasto - cfe.loc[tarifa,'LIV_pesos'])/cfe.loc[tarifa,'Excedente'] + cfe.loc[tarifa,'LimIV']
            else:
                kwh_v = (gasto - cfe.loc[tarifa,'LIVA_pesos'])/cfe.loc[tarifa,'Excedente'] + cfe.loc[tarifa,'LimIVA']
        else:
            kwh_v = (gasto - dac.loc[region,'c_fijo_v_m'])/dac.loc[region,'c_ener_m_sm']
            
    if tipo_tar:         
        if (kwh_nv+kwh_v)/2 > cfe.loc[tarifa,'Ldac']:
            return 'DAC'
        else:
            return '1X'
    else:
        return (kwh_nv+kwh_v)/2

In [ ]:
# Pesos to energy
def converter(gasto, tipo):
    

## Calculation

In [ ]:
ele['kwh'] = np.vectorize(mxn_to_kwh)(ele['gasto_tri_ele']/3, ele['Tarifa'], ele['Region']) 
ele['kwh/p'] = ele['kwh']/ele['tot_integ']
ele['tipo_tar'] = np.vectorize(mxn_to_kwh)(ele['gasto_tri_ele']/3, ele['Tarifa'], ele['Region'], tipo_tar=True) 